In [ ]:
# Import libraries
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import nltk
nltk.download('vader_lexicon')  # Download VADER lexicon for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
# Define model and tokenizer names (DistilBERT for sentiment analysis)
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
print("Model and tokenizer loaded successfully!")

In [ ]:
import onemkl as mkl

def clean_email(text):
  text = mkl.tolower(text)  # Convert to lowercase using oneMKL
  return text

In [ ]:
# Sentiment analysis using VADER
def predict_sentiment_vader(email):
  cleaned_email = clean_email(email)
  analyzer = SentimentIntensityAnalyzer()
  sentiment_scores = analyzer.polarity_scores(cleaned_email)
  # Extract sentiment category based on highest compound score
  if sentiment_scores['compound'] >= 0.05:
    return "Positive"
  elif sentiment_scores['compound'] <= -0.05:
    return "Negative"
  else:
    return "Neutral"

In [ ]:
# Sentiment analysis using pre-trained model 
import torch
def predict_sentiment_distilbert(email):
  # Tokenize the email
  tokenized_email = tokenizer(email, padding="max_length", truncation=True)

  # Get input IDs and attention mask from the tokenized email
  input_id = torch.tensor(tokenized_email["input_ids"], dtype=torch.long)  # Assuming input_ids are integers
  attention_mask = torch.tensor(tokenized_email["attention_mask"], dtype=torch.long)  # Assuming attention_mask are integers


  # Forward pass through the model
  outputs = model(input_id, attention_mask=attention_mask)
  # Get the predicted sentiment category (argmax of logits)
  predicted_sentiment = torch.argmax(outputs.logits).detach().item()

  # Define sentiment labels based on the model's output 
  sentiment_labels = {0: "Negative", 1: "Neutral", 2: "Positive"}

  return sentiment_labels[predicted_sentiment]

# Define reply suggestions based on sentiment
reply_suggestions = {
  "Negative": "I'm sorry to hear about the difficulties that you faced. Is there anything I can do to help resolve this issue?",
  "Neutral": "Thank you for bringing this to my attention.I acknowledge the information you've provided.",
  "Positive": "Thank you for your email and the positive update. I appreciate you keeping me informed."
}

In [ ]:
# Test the prediction functions with a sample email
sample_email = input("Enter the content of your Mail: ")

In [ ]:
# Sentiment analysis using VADER
vader_sentiment = predict_sentiment_vader(sample_email)
print(f"\nPredicted Sentiment (VADER) for '{sample_email}': {vader_sentiment}")

In [ ]:
# Sentiment analysis using DistilBERT
distilbert_sentiment = predict_sentiment_distilbert(sample_email)
print(f"\nPredicted Sentiment (DistilBERT) for '{sample_email}': {distilbert_sentiment}")

In [ ]:
# Choose the reply suggestion based on the preferred sentiment analysis method
# You can choose VADER or DistilBERT based on your needs
print("1.",vader_sentiment,"2.",distilbert_sentiment)
Choice=input("Enter your choice: ")
if (Choice=='1'):
  predicted_sentiment = vader_sentiment  # Or distilbert_sentiment
else:
  predicted_sentiment = distilbert_sentiment
reply_suggestion = reply_suggestions.get(predicted_sentiment)

In [ ]:
print(f"\nSuggested Reply: {reply_suggestion}")